# Day 7 Lecture Break/Practice Solutions

### Getting Forecast from OpenWeather


In [ ]:
import requests
import json
import pandas as pd
from datetime import datetime


def get_forecast(lat, lon, units='imperial', api_key='d36fa352ac73226b30772f64675f41bb'):
    """ creates a function that gets the hourly forecast (for the next two days) for a given lat and lon
    
    Args:
        lat (float): latitude
        lon (float): longitude
        units (str): units
        api_key (str): key for accessing API
        
    Returns:
        df_hourly (data frame): data frame with rows corresponding to hours and columns various features
    
    """
    # build url
    url = f'https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&appid={api_key}&units={units}'

    # get url response as a string
    url_text = requests.get(url).text

    # convert from json to a nested dict
    weather_dict = json.loads(url_text)

    # build dataframe from nested dict
    df_hourly = pd.DataFrame()
    for hour_dict in weather_dict['hourly']:
        # we add a 'datetime' feature from the unix time (easier to use)
        hour_dict['datetime'] = datetime.fromtimestamp(hour_dict['dt'])

        # build a single row of dataframe (weather prediction for some hour)
        s_hour = pd.Series(hour_dict)

        # collect forecasts in a dataframe
        df_hourly = df_hourly.append(s_hour, ignore_index=True)

    return df_hourly

### Converting Unix to Some Timezone

In [ ]:
def from_unix_to_datetime(unix_time, timezone_to='US/Eastern'):
    """ converts unix time to a datetime object
    
    Args:
        unix_time (float): unix time (sec since jan 1 1970 UTC)
        timezone_to (str): timezone of output datetime
        
    Returns:
        datetime_tz (datetime): datetime object corresponding to 
            unix_time
    """

    # build datetime (by default, has no timezone)
    datetime_from = datetime.utcfromtimestamp(unix_time)

    # add timezone to datetime
    tz_from = pytz.timezone('UTC')
    datetime_from = tz_from.localize(datetime_from)

    # Convert time zone
    tz_to = pytz.timezone(timezone_to)
    datetime_tz = datetime_from.astimezone(tz_to)

    return datetime_tz